In [2]:
import pandas as pdh
import numpy as np

### Load the NCC Data

In [3]:
ncc = pd.read_csv('../data/NCC_2018_nutrients_per_100g_originalcolnames.txt',sep='\t')

Set Food ID as the index

In [5]:
ncc['Food ID'].is_unique

True

In [6]:
ncc = ncc.set_index('Food ID')

### Load the data from the ASA24 Recalls

In [7]:
asa_24 = pd.read_csv('../data/training_for_GS_122118.csv')

Set FoodCode as the index 

In [9]:
asa_24 = asa_24.set_index('FoodCode')

### Load the file listing matching columns between the datasets

In [11]:
matching = pd.read_csv('../data/matching_ncc_fndds_columns.txt',sep='\t')

Get the list of columns for each dataset

In [36]:
ncc_cols = matching['NCC.Term'].values.tolist()
asa_24_cols = matching['FNDDS.Term'].values.tolist()
asa_24_cols = [val.replace(" ","") for val in asa_24_cols]

### Calculate the pairwise correlations
Define a function to calculate the pairwise PCC matrix between two matrices A and B

In [15]:
def row_corr(A,B):
    #number of columns in A or B
    N = B.shape[1]

    # Store row-wise sums of A and B, as they would be used at few places
    sA = A.sum(1)
    sB = B.sum(1)

    # Compute the four terms in pcc matrix-wise
    p1 = N*np.einsum('ik,jk->ij',A,B)
    p2 = sB*sA[:,None]
    p3 = N*((B**2).sum(1)) - (sB**2)
    p4 = N*((A**2).sum(1)) - (sA**2)

    # compute pcc as 2D array 
    pcorr = ((p1 - p2)/np.sqrt(p3*p4[:,None]))
    return pcorr

Get the columns provided by the `matching` file

In [41]:
A = asa_24.loc[:,asa_24_cols].values
B = ncc.loc[:,ncc_cols].values

In [42]:
corr = row_corr(A,B)

/software/anaconda3/4.5.12/lssc0-linux/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/software/anaconda3/4.5.12/lssc0-linux/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [48]:
indices = np.nanargmax(corr,axis=1)

In [68]:
lookup_argmax = ncc.iloc[indices,:]

In [69]:
results_argmax = pd.DataFrame()
results_argmax['asa_24'] = asa_24['Food_Description'].values
results_argmax['matched'] = lookup_argmax['Food Description'].values

In [70]:
results_argmax

,asa_24,matched
0,"Milk, condensed, sweetened (formerly NS as to ...","milk, condensed (sweetened), regular, regular"
1,"Cocoa, whey, and low-calorie sweetener mixture...","milk, goat"
2,"Eggnog, made with whole milk","cream, half & half, lowfat"
3,"Instant breakfast, powder, not reconstituted","special formulated products, bars or wafers, e..."
4,"Pudding, chocolate, ready-to-eat, low calorie,...","mixed dish, brand name listing, Jenny Craig, J..."
5,Custard,"desserts - miscellaneous, pudding, tapioca pud..."
6,"Pudding, canned, chocolate, reduced fat","milk, mixtures and milk drinks, cocoa or hot c..."
7,"Pudding, canned, flavors other than chocolate,...","milk, mixtures and milk drinks, cocoa or hot c..."
8,"Cheese, cottage, lowfat, with fruit","salad, cottage cheese, with gelatin, with fruit"
9,"Cheese, processed, American or Cheddar type","cheese, brand name listing, Kraft, Kraft Sprea..."
